In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,dense_rank,sum,avg
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

24/10/17 22:50:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
Drivers = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Drivers") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Vehicles = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Vehicles") \
    .option("user", "root") \
    .option("password", "password") \
    .load()


Trips = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Trips") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [15]:
Trips\
    .join(Vehicles,on=Trips.vehicle_id==Vehicles.vehicle_id,how="inner")\
    .join(Drivers,on=Vehicles.driver_id==Drivers.driver_id,how="inner")\
    .groupBy(Drivers.driver_id,col("fuel_type"))\
    .agg(avg("rating").alias("rating"),
         sum("distance").alias("distance"),
         sum("accidents").alias("total_accidents"))\
    .withColumn("drnk",dense_rank()\
                .over(Window\
                      .partitionBy("fuel_type")\
                      .orderBy(col("rating").desc(),col("distance").desc(),col("total_accidents"))))\
    .filter(col("drnk")==1)\
    .drop("drnk","total_accidents")\
    .show()

+---------+---------+------+--------+
|driver_id|fuel_type|rating|distance|
+---------+---------+------+--------+
|        2| Electric|   4.5|     180|
|        3| Gasoline|   5.0|     100|
+---------+---------+------+--------+

